In [ ]:
!pip install h2o

     |████████████████████████████████| 176.0 MB 5.5 kB/s 
  Created wheel for h2o: filename=h2o-3.36.0.2-py2.py3-none-any.whl size=176112127 sha256=95ad4913e3234d0491715b9799b0ffd98e4c70c96addd9f75770a61d974ded14
  Stored in directory: /root/.cache/pip/wheels/0b/83/bd/343a6140119b4098103e725e1cfe9f198568d13cf103acbc7c
Successfully built h2o


In [ ]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,56 mins 24 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_w1aard
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.883 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
train2 = h2o.import_file("train2_1.csv")[:,1:]#pd.read_csv('train2_1.csv').iloc[:, 1:]
test2 = h2o.import_file("test2_1.csv")[:,1:] # pd.read_csv('test2_1.csv').iloc[:, 1:]
# test3.isna().sum()
train2.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


amount_tsh_clip,date_recorded_year,date_recorded_season,gps_height_group,basin,region,population,public_meeting,permit,construction_year_group,extraction_type_class,management,payment_type,quality_group,quantity,source,waterpoint_type,longitude_cat,latitude_band,age,lga_clip,ward_clip,status_group
500,2011,1,>=850,Rufiji,Iringa,80,True,False,>=1997,handpump,vwc,monthly,good,enough,hand dtw,hand pump,1,4,3,Njombe,Others,1
250,2011,2,<80,Pangani,Tanga,50,True,True,1960-1997,gravity,vwc,never pay,colored,enough,spring,communal standpipe,2,8,40,Others,Maramba,0
250,2011,3,<80,Lake Victoria,Mwanza,0,True,False,,handpump,vwc,annually,good,enough,shallow well,hand pump,0,1,15,Others,Others,0
250,2011,3,>=850,Lake Victoria,Mwanza,500,True,True,>=1997,handpump,vwc,never pay,good,insufficient,shallow well,hand pump,0,1,2,Others,Others,1
500,2013,2,>=850,Pangani,Kilimanjaro,250,True,False,1960-1997,gravity,wua,never pay,good,insufficient,spring,communal standpipe,2,7,31,Others,Others,1
250,2011,2,>=850,Lake Nyasa,Iringa,40,True,False,>=1997,gravity,vwc,on failure,good,dry,river,communal standpipe,1,4,7,Ludewa,Others,1
6,2013,3,>=850,Pangani,Kilimanjaro,20,True,True,>=1997,gravity,water board,per bucket,good,enough,spring,communal standpipe,2,7,5,Rural,Others,1
10,2013,1,>=850,Internal,Manyara,200,True,True,>=1997,gravity,vwc,per bucket,good,insufficient,spring,communal standpipe,1,5,2,Others,Others,0
250,2013,1,<80,Lake Victoria,Shinyanga,0,True,False,,handpump,wug,never pay,good,enough,shallow well,hand pump,0,2,15,Maswa,Others,1
250,2011,1,80-850,Wami / Ruvu,Morogoro,45,True,True,1960-1997,gravity,vwc,never pay,good,insufficient,river,communal standpipe,2,7,27,Mvomero,Others,1


In [ ]:
train, test = train2.split_frame(ratios=[.7])

x = train.columns
y = 'status_group'
x.remove(y)

train[y] = train[y].asfactor()
# test[y] = test[y].asfactor()

keep = ['amount_tsh_clip', 'date_recorded_year', 'date_recorded_season', 'gps_height_group',
        'basin', 'region', 'population', 'public_meeting', 'permit', 'construction_year_group',
        'extraction_type_class', 'management', 'payment_type', 'quality_group', 'quantity', 'source', 
        'waterpoint_type','longitude_cat', 'latitude_band', 'age', 'lga_clip', 'ward_clip']
contFeature = ['amount_tsh_clip', 'date_recorded_year', 'population', 'age']
catFeature = [i for i in keep if i not in contFeature]

for i in catFeature:
  train[i] = train[i].asfactor()
  test2[i] = test2[i].asfactor()

# Run AutoML for 60 seconds
aml = H2OAutoML(max_runtime_secs = 360)
aml.train(x = x, y = y, training_frame = train, validation_frame = test)



AutoML progress: |
18:03:54.536: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_10_20220213_180354

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09998961676549199
RMSE: 0.3162113482553907
LogLoss: 0.3203622024938898
Null degrees of freedom: 9789
Residual degrees of freedom: 9784
Null deviance: 13507.05729811117
Residual deviance: 6272.691924830362
AIC: 6284.691924830362
AUC: 0.9405915081388302
AUCPR: 0.9394794312419815
Gini: 0.8811830162776604

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.42558858295067276: 


,,0,1,Error,Rate
0,0,4621.0,674.0,0.1273,(674.0/5295.0)
1,1,697.0,3798.0,0.1551,(697.0/4495.0)
2,Total,5318.0,4472.0,0.14,(1371.0/9790.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.425589,0.847106,207.0
1,max f2,0.253530,0.892424,272.0
2,max f0point5,0.601830,0.884337,145.0
3,max accuracy,0.474317,0.862921,188.0
4,max precision,0.999664,1.000000,0.0
5,max recall,0.034662,1.000000,381.0
6,max specificity,0.999664,1.000000,0.0
7,max absolute_mcc,0.474317,0.724514,188.0
8,max min_per_class_accuracy,0.404649,0.855617,214.0
9,max mean_per_class_accuracy,0.459730,0.859003,193.0



Gains/Lift Table: Avg response rate: 45.91 %, avg score: 45.44 %


...

,,0,1,Error,Rate
0,0,14598.0,3473.0,0.1922,(3473.0/18071.0)
1,1,3124.0,11962.0,0.2071,(3124.0/15086.0)
2,Total,17722.0,15435.0,0.199,(6597.0/33157.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.414409,0.783854,213.0
1,max f2,0.170136,0.849135,317.0
2,max f0point5,0.691345,0.831579,117.0
3,max accuracy,0.520197,0.807522,174.0
4,max precision,0.999674,0.994368,0.0
5,max recall,0.000666,1.000000,399.0
6,max specificity,0.999674,0.999613,0.0
7,max absolute_mcc,0.567252,0.615556,157.0
8,max min_per_class_accuracy,0.405239,0.798621,216.0
9,max mean_per_class_accuracy,0.458391,0.801543,196.0



Gains/Lift Table: Avg response rate: 45.50 %, avg score: 45.50 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010013,0.999841,2.191245,2.191245,0.996988,0.999877,0.996988,0.999877,0.021941,0.021941,119.124549,119.124549,0.021886
1,2,0.020026,0.999729,2.184625,2.187935,0.993976,0.999790,0.995482,0.999833,0.021875,0.043815,118.462542,118.793546,0.043649
2,3,0.030009,0.999534,2.184585,2.186821,0.993958,0.999642,0.994975,0.999770,0.021808,0.065624,118.458542,118.682102,0.065347
3,4,0.040022,0.995629,2.184625,2.186272,0.993976,0.998525,0.994725,0.999458,0.021875,0.087498,118.462542,118.627171,0.087111
4,5,0.050005,0.992545,2.164665,2.181958,0.984894,0.993919,0.992762,0.998352,0.021609,0.109108,116.466518,118.195822,0.108444
5,6,0.100009,0.981257,2.184609,2.183284,0.993969,0.987031,0.993366,0.992692,0.109240,0.218348,118.460945,118.328383,0.217131
6,7,0.150014,0.957556,2.152795,2.173121,0.979493,0.971197,0.988741,0.985527,0.107649,0.325998,115.279474,117.312080,0.322899
7,8,0.200018,0.888138,2.073258,2.148155,0.943305,0.928026,0.977382,0.971152,0.103672,0.429670,107.325799,114.815510,0.421369
8,9,0.299997,0.674653,1.747021,2.014471,0.794872,0.786367,0.916558,0.909569,0.174665,0.604335,74.702135,101.447074,0.558405
9,10,0.400006,0.500278,1.226854,1.817552,0.558203,0.581260,0.826962,0.827486,0.122697,0.727032,22.685439,81.755180,0.600033


In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_10_20220213_180354,0.887966,0.407822,0.888015,0.199633,0.363325,0.132005
StackedEnsemble_BestOfFamily_2_AutoML_10_20220213_180354,0.887552,0.408685,0.887699,0.201967,0.363597,0.132203
StackedEnsemble_BestOfFamily_1_AutoML_10_20220213_180354,0.885071,0.41243,0.88508,0.200549,0.365991,0.13395
GBM_4_AutoML_10_20220213_180354,0.884875,0.412568,0.885132,0.202431,0.365749,0.133773
GBM_1_AutoML_10_20220213_180354,0.884654,0.413226,0.884684,0.201754,0.366319,0.13419
DRF_1_AutoML_10_20220213_180354,0.884554,0.426483,0.884345,0.203483,0.366305,0.134179
GBM_3_AutoML_10_20220213_180354,0.88118,0.419165,0.881195,0.205665,0.369003,0.136164
GBM_2_AutoML_10_20220213_180354,0.879309,0.422614,0.87937,0.209676,0.370677,0.137401
XGBoost_2_AutoML_10_20220213_180354,0.877205,0.425691,0.877413,0.213341,0.372269,0.138584
XGBoost_1_AutoML_10_20220213_180354,0.876197,0.428122,0.876438,0.217306,0.373614,0.139587


In [ ]:
m = aml.get_best_model(criterion="logloss")
preds = m.predict(test2)
preds['p1']

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.7/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'ward_clip' has levels not trained on: ["Ifinga", "Kemambo", "Kitete", "Matarawe", "Nsemulwa", "Tunduma", "Ukata"]
  warnings.warn(w)


p1
0.0143235
0.993131
0.0811992
0.0596556
0.427825
0.0140678
0.132277
0.366838
0.403304
0.450881


In [ ]:
import gdown
import pandas as pd
# Download file with features in the test set
testFeaturesURL = "https://drive.google.com/uc?id=1C64MAXqe4R4UjFBJcyK7hmKcNIVj3C8x"
gdown.download(testFeaturesURL, "test_features.csv", quiet=True)

# Read test set features into a pandas data frame
testFeatures = pd.read_csv("test_features.csv")
testFeatures.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,47521,6000.0,3/14/2011,Roman,1390,Roman,34.934429,-9.859748,none,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,47522,0.0,1/28/2013,Unicef,263,UNICEF,38.481475,-11.147901,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
2,47523,0.0,10/14/2012,World Vision,0,World vision,33.802775,-3.291305,Pamba,Internal,Nkilifa,Shinyanga,17,2,Maswa,Busilili,0,True,GeoData Consultants Ltd,WUG,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,other,other,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
3,47524,0.0,3/15/2013,Lawatefuka Water Supply,1368,Lawatefuka water sup,37.086057,-3.181949,Kwa John Izack Mmari,Pangani,Omarini,Kilimanjaro,3,7,Siha,Siha Kaskazini,1,True,GeoData Consultants Ltd,Water Board,BL Bondeni,True,2009,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
4,47525,0.0,10/20/2012,Biore,0,WEDECO,34.365589,-3.626754,Mwabasabi,Internal,Mwabasabi,Shinyanga,17,6,Meatu,Nkoma,0,True,GeoData Consultants Ltd,WUG,None,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [ ]:
pre = preds['p1'].as_data_frame()['p1']
pre

my_submission = pd.DataFrame({'id': testFeatures.id, 'predicted': pre})
my_submission

my_submission.to_csv("submission.csv", index = False)